# [12 Python Decorators to Take Your Code to the Next Level](https://towardsdatascience.com/12-python-decorators-to-take-your-code-to-the-next-level-a910a1ab3e99)

## 1 — @logger (to get started)✏️

Если вы впервые знакомы с декораторами, то можете считать их функциями, которые принимают функции на вход и расширяют их функциональные возможности без изменения их основного назначения.

Давайте начнем с простого декоратора, который расширяет функцию, регистрируя время начала и окончания ее выполнения.

Результат работы декорируемой функции будет выглядеть следующим образом:

# ----- some_function: start -----
# some_function executing
# ----- some_function: end -----
Чтобы написать этот дешифратор, сначала нужно выбрать подходящее имя: назовем его logger.

Логгер - это функция, которая принимает функцию на вход и возвращает функцию на выход. Выходная функция обычно является расширенной версией входной. В нашем случае мы хотим, чтобы выходная функция окружала вызов входной функции утверждениями start и end.

Поскольку мы не знаем, какие аргументы использует входная функция, мы можем передать их из функции-обертки с помощью выражений *args и **kwargs. Эти выражения позволяют передавать произвольное количество позиционных и ключевых аргументов.

Вот простая реализация декоратора регистратора:

In [8]:

def logger(function):
    def wrapper(*args, **kwargs):
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output
    return wrapper

@logger
def some_function(text):
    print(text)

some_function("first test")
# ----- some_function: start -----
# first test
# ----- some_function: end -----
some_function("second test")
# ----- some_function: start -----
# second test
# ----- some_function: end -----


----- some_function: start -----
first test
----- some_function: end -----
----- some_function: start -----
second test
----- some_function: end -----


## 2 — @wraps 🎁
Этот декоратор обновляет функцию-обертку, чтобы она выглядела как оригинальная функция и наследовала ее имя и свойства.

Чтобы понять, что делает @wraps и почему вы должны его использовать, давайте возьмем предыдущий декоратор и применим его к простой функции, которая складывает два числа.

In [9]:
def logger(function):
    def wrapper(*args, **kwargs):
        """wrapper documentation"""
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output
    return wrapper

@logger
def add_two_numbers(a, b):
    """this function adds two numbers"""
    return a + b

In [10]:
print(add_two_numbers.__name__)

print(add_two_numbers.__doc__)

wrapper
wrapper documentation


Если мы проверим имя и документацию декорированной функции add_two_numbers, вызвав атрибуты `__name__` и `__doc__`, мы получим ... неестественные (и все же ожидаемые) результаты:

Вместо этого мы получаем имя обертки и документацию ⚠️.

Это нежелательный результат. Мы хотим сохранить имя и документацию исходной функции. Вот тут-то и пригодится декоратор @wraps.

Все, что вам нужно сделать, это украсить функцию-обертку.

In [11]:
from functools import wraps

def logger(function):
    @wraps(function)
    def wrapper(*args, **kwargs):
        """wrapper documentation"""
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output
    return wrapper

@logger
def add_two_numbers(a, b):
    """this function adds two numbers"""
    return a + b

In [12]:
print(add_two_numbers.__name__)

print(add_two_numbers.__doc__)

add_two_numbers
this function adds two numbers


## 3 — @lru_cache 💨

Это встроенный декоратор, который можно импортировать из functools.

Он кэширует возвращаемые значения функции, используя алгоритм наименее часто используемых значений (LRU) для отбрасывания наименее используемых значений, когда кэш заполняется.

Обычно я использую этот декоратор для длительно выполняющихся задач, которые не меняют результат при одинаковых входных данных, например, запрос к базе данных, запрос статической удаленной веб-страницы или тяжелая обработка.

В следующем примере я использую lru_cache для декорирования функции, которая имитирует некоторую обработку. Затем я применяю функцию на одном и том же входе несколько раз подряд.

In [13]:
import random
import time
from functools import lru_cache


@lru_cache(maxsize=None)
def heavy_processing(n):
    sleep_time = n + random.random()
    time.sleep(sleep_time)

# first time
%%time
heavy_processing(0)
# CPU times: user 363 µs, sys: 727 µs, total: 1.09 ms
# Wall time: 694 ms

# second time
%%time
heavy_processing(0)
# CPU times: user 4 µs, sys: 0 ns, total: 4 µs
# Wall time: 8.11 µs

# third time
%%time
heavy_processing(0)
# CPU times: user 5 µs, sys: 1 µs, total: 6 µs
# Wall time: 7.15 µs

UsageError: Line magic function `%%time` not found.


In [14]:
%%time
heavy_processing(1)

CPU times: user 1.19 ms, sys: 1.65 ms, total: 2.83 ms
Wall time: 1.95 s


In [15]:
%%time
heavy_processing(1)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 3.81 µs


## 4 — @repeat 🔁
Этот декоратор заставляет функцию вызываться несколько раз подряд.

Это может быть полезно для отладки, стресс-тестов или автоматизации повторения нескольких задач.

В отличие от предыдущих декораторов, этот ожидает входной параметр.

In [16]:
def repeat(number_of_times):
    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(number_of_times):
                func(*args, **kwargs)
        return wrapper
    return decorate

В следующем примере определяется декоратор repeat, который принимает в качестве аргумента число раз. Затем декоратор определяет функцию wrapper, которая оборачивается вокруг декорируемой функции. Функция-обертка вызывает декорируемую функцию количество раз, равное указанному числу.

In [17]:
@repeat(5)
def dummy():
    print("hello")

dummy()

hello
hello
hello
hello
hello


## 5 — @timeit ⏲️
Этот декоратор измеряет время выполнения функции и выводит результат: это служит для отладки или мониторинга.

В следующем фрагменте декоратор timeit измеряет время выполнения функции process_data и выводит прошедшее время в секундах.

In [18]:
import time
from functools import wraps

def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds to complete')
        return result
    return wrapper

@timeit
def process_data():
    time.sleep(1)

process_data()
# process_data took 1.000012 seconds to complete

process_data took 1.005098 seconds to complete


## 6 — @retry 🔁
Этот декоратор заставляет функцию повторять попытку несколько раз, когда она сталкивается с исключением.

Он принимает три аргумента: `количество повторных попыток, исключение, которое нужно поймать и повторить, и время сна между повторными попытками`.

Это работает следующим образом:

Функция-обертка запускает цикл for с количеством итераций num_retries.
На каждой итерации она вызывает входную функцию в блоке try/except. При успешном вызове она прерывает цикл и возвращает результат. В противном случае он спит в течение sleep_time секунд и переходит к следующей итерации.
Если после завершения цикла for вызов функции не удается, функция-обертка вызывает исключение.

In [19]:
import random
import time
from functools import wraps

def retry(num_retries, exception_to_check, sleep_time=0):
    """
    Decorator that retries the execution of a function if it raises a specific exception.
    """
    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(1, num_retries+1):
                try:
                    return func(*args, **kwargs)
                except exception_to_check as e:
                    print(f"{func.__name__} raised {e.__class__.__name__}. Retrying...")
                    if i < num_retries:
                        time.sleep(sleep_time)
            # Raise the exception if the function was not successful after the specified number of retries
            raise e
        return wrapper
    return decorate

@retry(num_retries=2, exception_to_check=ValueError, sleep_time=5)
def random_value():
    value = random.randint(1, 5)
    if value == 3:
        raise ValueError("Value cannot be 3")
    return value

random_value()
# random_value raised ValueError. Retrying...
# 1

random_value()
# 5

2

## 7 — @countcall 🔢
Этот декоратор подсчитывает, сколько раз была вызвана функция.

Это число хранится в атрибуте обертки count .

In [20]:
from functools import wraps

def countcall(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        result = func(*args, **kwargs)
        print(f'{func.__name__} has been called {wrapper.count} times')
        return result
    wrapper.count = 0
    return wrapper

@countcall
def process_data():
    pass

process_data()
# process_data has been called 1 times
process_data()
# process_data has been called 2 times
process_data()
# process_data has been called 3 times

process_data has been called 1 times
process_data has been called 2 times
process_data has been called 3 times


# 8 — @rate_limited 🚧

Это декоратор, который ограничивает частоту вызова функции, откладывая ее на некоторое время, если функция вызывается слишком часто.

In [21]:
import time
from functools import wraps

def rate_limited(max_per_second):
    min_interval = 1.0 / float(max_per_second)
    def decorate(func):
        last_time_called = [0.0]
        @wraps(func)
        def rate_limited_function(*args, **kargs):
            elapsed = time.perf_counter() - last_time_called[0]
            left_to_wait = min_interval - elapsed
            if left_to_wait > 0:
                time.sleep(left_to_wait)
            ret = func(*args, **kargs)
            last_time_called[0] = time.perf_counter()
            return ret
        return rate_limited_function
    return decorate

The decorator works by measuring the time elapsed since the last call to the function and waiting for an appropriate amount of time if necessary to ensure that the rate limit is not exceeded. The waiting time is calculated as min_interval - elapsed, where min_interval is the minimum time interval (in seconds) between two function calls and elapsed is the time elapsed since the last call.

If the time elapsed is less than the minimum interval, the function waits for left_to_wait seconds before being executed again.

__This function hence introduces a slight time overhead between the calls but ensures that the rate limit is not exceeded.__



In [22]:
# There’s also a third-party package that implements API rate limit: it’s called ratelimit.

%pip install ratelimit
%pip install requests

from ratelimit import limits

import requests

FIFTEEN_MINUTES = 900

@limits(calls=15, period=FIFTEEN_MINUTES)
def call_api(url):
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception('API response: {}'.format(response.status_code))
    return response

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


If the decorated function is called more times than allowed a `ratelimit.RateLimitException` is raised.

To be able to handle this exception, you can use the `sleep_and_retry` decorator in combination with the `ratelimit` decorator.

In [23]:
@sleep_and_retry
@limits(calls=15, period=FIFTEEN_MINUTES)
def call_api(url):
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception('API response: {}'.format(response.status_code))
    return response

NameError: name 'sleep_and_retry' is not defined

# 9 — @dataclass 🗂️
The `@dataclass` decorator in Python is used to decorate classes.

It automatically generates special methods such as `__init__, __repr__, __eq__, __lt__, and __str__` for classes that primarily store data. This can reduce the boilerplate code and make the classes more readable and maintainable.

It also provides nifty methods off-the-shelf to represent objects nicely, convert them into JSON format, make them immutable, etc.

The `@dataclass` decorator was introduced in Python 3.7 and is available in the standard library.



In [31]:
from dataclasses import dataclass, asdict

@dataclass
class Person:
    first_name: str
    last_name: str
    age: int
    job: str

    def __eq__(self, other):
        if isinstance(other, Person):
            return self.age == other.age
        return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Person):
            return self.age < other.age
        return NotImplemented


john = Person(first_name="John", 
              last_name="Doe", 
              age=30, 
              job="doctor",)

anne = Person(first_name="Anne", 
              last_name="Smith", 
              age=40, 
              job="software engineer",)

print(john == anne)
# False

print(anne > john)
# True

asdict(anne)
#{'first_name': 'Anne',
# 'last_name': 'Smith',
# 'age': 40,
# 'job': 'software engineer'}

False
True


{'first_name': 'Anne',
 'last_name': 'Smith',
 'age': 40,
 'job': 'software engineer'}

# 10 — @register 🛑
Если ваш сценарий Python случайно завершается, а вы все еще хотите выполнить некоторые задачи для сохранения работы, очистки или печати сообщения, я нахожу, что декоратор register весьма удобен в этом контексте.

In [ ]:
from atexit import register

@register
def terminate():
    perform_some_cleanup()
    print("Goodbye!")

while True:
    print("Hello")

# When running this script and hitting CTRL+C, we see the output of the terminate function.

# 11 — @property 🏠
Декоратор свойств используется для определения свойств класса, которые по сути являются методами getter, setter и deleter для атрибута экземпляра класса.

Используя декоратор свойств, вы можете определить метод как свойство класса и получить к нему доступ, как к атрибуту класса, без явного вызова метода.

Это полезно, если вы хотите добавить некоторые ограничения и логику проверки вокруг получения и установки значения.

В следующем примере мы определяем сеттер для свойства rating, чтобы наложить ограничение на вводимое значение (от 0 до 5).

In [6]:
class Movie:
    def __init__(self, r):
        self._rating = r

    @property
    def rating(self):
        return self._rating

    @rating.setter
    def rating(self, r):
        if 0 <= r <= 5:
            self._rating = r
        else:
            raise ValueError("The movie rating must be between 0 and 5!")

batman = Movie(2.5)
print(batman.rating)
# 2.5
batman.rating = 4
print(batman.rating)
# 4
print(f'batman.rating = 10') # ValueError("The movie rating must be between 0 and 5!")

2.5
4
batman.rating = 10


# 12 — @singledispatch
Этот декоратор позволяет функции иметь различные реализации для разных типов аргументов.

In [7]:
from functools import singledispatch

@singledispatch
def fun(arg):
    print("Called with a single argument")

@fun.register(int)
def _(arg):
    print("Called with an integer")

@fun.register(list)
def _(arg):
    print("Called with a list")

fun(1)  # Prints "Called with an integer"
fun([1, 2, 3])  # Prints "Called with a list"

Called with an integer
Called with a list


## Заключение
Декораторы - это полезные абстракции для расширения вашего кода дополнительными функциями, такими как кэширование, автоматическое повторение, ограничение скорости, ведение журнала или превращение ваших классов в суперзаряженные контейнеры данных.

Но на этом все не заканчивается, поскольку вы можете быть более креативными и реализовать свои собственные декораторы для решения очень специфических проблем.

Вот список замечательных декораторов для вдохновения.

Спасибо за чтение!